In [1]:
--- Libraries:
arguments = require 'Settings.arguments'
constants = require 'Settings.constants'
card_to_string = require 'Game.card_to_string_conversion'
card_tools = require 'Game.card_tools'
game_settings = require 'Settings.game_settings'
Plot = require 'itorch.Plot'
nninit = require 'nninit'
require 'torch'
require 'math'
require 'Tree.tree_builder'
require 'Tree.tree_data_generation'
require 'Tree.tree_values'
require 'Tree.tree_visualiser'
require 'nn'
require 'Tree.tree_cfr'
require 'Tree.tree_cfr2'
require 'nngraph'
require 'optim'
require 'image'
require 'NN.strategy_net_builder'
require 'NN.strategy_net_trainer'
require 'Tree.tree_warm_start'
---require 'Adversarials.exploitability'
require 'Adversarials.exploitabilityVS'



In [2]:
function get_first_mask_tensor(index)
    local output = torch.range(1,6)
    if index ==1 then
        return torch.range(2,6)
    end
    local out1 = output:clone():narrow(1,1,index-1)
    local out2 = output:clone():narrow(1,index+1,6-index)
    local result = out1:cat(out2)
    return result
end

function get_second_mask_tensor(input_tensor,index)
    local result = {}
    local input = input_tensor:clone()
    for i = 1,input:size(1) do
        if input[i] ~= index then
            table.insert(result,input[i])
        end
    end
    return torch.Tensor(result)
end


function construct_possible_game_states()
    local states_tensor = torch.Tensor()
    local first_card_tensor = torch.Tensor({1,3,5})
    for i = 1,first_card_tensor:size(1) do
        local second_card_tensor = get_first_mask_tensor(first_card_tensor[i])
        for j =1, second_card_tensor:size(1) do
            local third_card_tensor = get_second_mask_tensor(second_card_tensor,second_card_tensor[j])
            for k = 1,third_card_tensor:size(1) do
                local states = torch.Tensor({first_card_tensor[i],second_card_tensor[j],third_card_tensor[k]}):view(1,3)
                states_tensor = states_tensor:cat(states,1)
            end
        end
    end
    return states_tensor
end

function distribute_cards(cards_tensor)
    local cards = {}
    cards.P1 = cards_tensor[1]
    cards.P2 = cards_tensor[2]
    cards.board = cards_tensor[3]
    return cards
end

In [3]:
--- Important variables:
pot = 200

In [4]:
--- Create the tree
builder = PokerTreeBuilder()
--- Parameters for the tree
params = {}
params.root_node = {}
params.root_node.board = card_to_string:string_to_board('')
params.root_node.street = 1
params.root_node.current_player = constants.players.P1
params.root_node.bets = arguments.Tensor{pot,pot}
--- CFR Solver
starting_ranges = arguments.Tensor(constants.players_count, game_settings.card_count)
starting_ranges[1]:copy(card_tools:get_uniform_range(params.root_node.board))
starting_ranges[2]:copy(card_tools:get_uniform_range(params.root_node.board))
--- exploitability
exploit = TreeValues()

In [5]:
--- Initialize trees
tree_bench = builder:build_tree(params)
tree_500 = builder:build_tree(params)
tree_200 = builder:build_tree(params)



In [6]:
---visualiser = TreeVisualiser()
---visualiser:graphviz(tree1, "zero_blind")

In [7]:
solver5 = TreeCFR2()
solver5.cfr_skip = 20
solver5:run_cfr(tree_500,starting_ranges,500)

In [8]:
solver2 = TreeCFR2()
solver2.cfr_skip = 20
solver2:run_cfr(tree_200,starting_ranges,50)

In [9]:
--- NNWarm Starting
dataNN = TreeData(tree_200)
dataNN:get_training_set(tree_200,1)

dataNN5 = TreeData(tree_500)
dataNN5:get_training_set(tree_500,1)



In [10]:
tree_eval_200 = StrategyEvaluator(tree_bench,tree_200)
tree_eval_500 = StrategyEvaluator(tree_bench,tree_500)
tree_eval = StrategyEvaluator(tree_200,tree_500)


In [24]:
tree_eval_200:play_all_combinations_n_times(tree_bench,tree_200,20)
tree_eval_200:display_results()

ties: 	67
P1 won: 	457
P2 won: 	676


In [22]:
tree_eval_500:play_all_combinations_n_times(tree_bench,tree_500,20)
tree_eval_500:display_results()

ties: 	61
P1 won: 	440
P2 won: 	699


In [23]:
tree_eval:play_all_combinations_n_times(tree_200,tree_500,50)
tree_eval:display_results()

ties: 	407
P1 won: 	1253
P2 won: 	1340


In [49]:
avg_pot_won = tree_eval:calculate_return(tree_200,tree_500,500,50)

iteration: 	1
Total number of pots won : 	0.2


iteration: 	50
Total number of pots won : 	0.0026666666666667


iteration: 	100
Total number of pots won : 	0.0093333333333333


iteration: 	150
Total number of pots won : 	0.0077777777777778


iteration: 	200
Total number of pots won : 	0.020166666666667


...

iteration: 	300
Total number of pots won : 	0.013777777777778


iteration: 	350
Total number of pots won : 	0.024619047619048


iteration: 	400
Total number of pots won : 	0.0051666666666667


iteration: 	450
Total number of pots won : 	0.012814814814815


iteration: 	500
Total number of pots won : 	0.023766666666667


In [32]:
---timer = torch.Timer()
--print('Time elapsed ' .. timer:time().real .. ' seconds')

In [47]:
avg_pot_won2 = tree_eval_200:calculate_return(tree_bench,tree_200,200,50)

iteration: 

	1
Total number of pots won : 	0.93333333333333


iteration: 	50
Total number of pots won : 	0.66233333333333


iteration: 	100
Total number of pots won : 	0.716


iteration: 	150
Total number of pots won : 	0.66744444444444


iteration: 	200
Total number of pots won : 	0.73283333333333


In [44]:
avg_pot_won5 = tree_eval_500:calculate_return(tree_bench,tree_500,200,50)

iteration: 	1
Total number of pots won : 	0.76666666666667


iteration: 	50
Total number of pots won : 	0.62333333333333


iteration: 	100
Total number of pots won : 	0.6705


iteration: 	150
Total number of pots won : 	0.69944444444444


iteration: 	200
Total number of pots won : 	0.69008333333333


In [48]:
local plot = Plot()
local x = tree_eval_200.range_games

plot:line(x,x:clone():fill(avg_pot_won2:mean()),'red' ,'Original CFR in the paper')
plot:line(x,avg_pot_won2:view(-1),'brown' ,'No Warm start CFR')

plot:line(x,x:clone():fill(avg_pot_won5:mean()),'blue' ,'Original CFR in the paper')
plot:line(x,avg_pot_won5:view(-1),'black' ,'No Warm start CFR')

plot:draw()



In [50]:
tree_eval:display_results()

ties: 	4013
P1 won: 	12608
P2 won: 	13379



In [61]:
tree_eval.A2_pot_matrix:sum()

142600


In [65]:
tree_eval.A2_pot_tensor:sum()/(60*500)

0.43333333333333


In [51]:
local plot = Plot()
local x = tree_eval.range_games

plot:line(x,x:clone():fill(avg_pot_won:mean()),'red' ,'Original CFR in the paper')
plot:line(x,avg_pot_won:view(-1),'brown' ,'No Warm start CFR')

plot:draw()



In [37]:
print(tree_eval.A2_pot_matrix:sum())
print(tree_eval.A2_pot_matrix:std())

ties: 	1587
P1 won: 	5059
P2 won: 	5354



12200
451.32317405549


In [ ]:
print("Total number of pots won : ",tree_eval.A2_pot_matrix:sum()/min_pot)

In [ ]:
print("average amount won per game",tree_eval.A2_pot_matrix:sum()/(number_of_times*60))

In [ ]:
avg_per_hand = tree_eval.A2_pot_matrix:mean(2)

In [ ]:
plot = Plot():circle(torch.range(1,avg_per_hand:size(1)),avg_per_hand:view(-1))
plot:legend(true):title('Player 2 pot distribution, 10000 games, each possible card dealt'):draw()


In [ ]:
-- Building the neural net model
strategy_nn = StrategyNN()
nn_model = strategy_nn.model
-- Building trainer:
nn_trainer = NNTrainer(tree_data800,nn_model)
-- Options:
opt = {}
opt.learningRate = 0.001
opt.momentum = 0.95
opt.validate = false

In [ ]:
cards = {}
cards.P1 = 1
cards.P2 = 2
cards.board = 4
number_of_games = 100

In [ ]:
tree_eval.A2_pot_tensor

In [ ]:
print(tree_eval.A2_pot_tensor:mean())
print(tree_eval.A2_pot_tensor:std())

In [ ]:
tree_eval:display_results()

In [ ]:
local pot_table = {}
local size_sample = 50
for i = 1,50 do
    tree_eval:play_several_games(tree1,tree2,size_sample)
    table.insert(pot_table,tree_eval.A2_pot/size_sample)
end
print("player_2 won: ",torch.FloatTensor(pot_table):mean())

In [ ]:
pot_table_A2 = {}
local number_of_games = 1000
local cards_tensor = construct_possible_game_states()
for i = 1,cards_tensor:size(1) do
    local cards = distribute_cards(cards_tensor[i])
    tree_eval:play_deterministic_games(tree1,tree2,number_of_games,cards)
    table.insert(pot_table_A2,tree_eval.A2_pot)
end
print("player_2 won: ",torch.FloatTensor(pot_table_A2):mean())
print("player_2 std: ",torch.FloatTensor(pot_table_A2):std())
tree_eval:display_results()

In [ ]:
torch.sqrt(60)*(1803/150716)

In [ ]:
plot = Plot():histogram(torch.FloatTensor(pot_table_A2)):draw()

In [ ]:
pot_table1 = {}
local number_of_games = 10
local cards_tensor = construct_possible_game_states()
for i = 1,cards_tensor:size(1) do
    for j = 1,number_of_games do
        local cards = distribute_cards(cards_tensor[i])
        tree_eval:play_deterministic_games(tree1,tree2,number_of_plays,cards)
        table.insert(pot_table1,tree_eval.A2_pot)
    end
end
print("player_2 won: ",torch.FloatTensor(pot_table1):mean())
print("player_2 std: ",torch.FloatTensor(pot_table1):std())

In [ ]:
pot_table1 = {}
local number_of_games = 20
local cards_tensor = construct_possible_game_states()
for i = 1,cards_tensor:size(1) do
    for j = 1,number_of_games do
        local cards = distribute_cards(cards_tensor[i])
        tree_eval:play_deterministic_games(tree1,tree2,number_of_plays,cards)
        table.insert(pot_table1,tree_eval.A2_pot)
    end
end
print("player_2 won: ",torch.FloatTensor(pot_table1):mean())
print("player_2 std: ",torch.FloatTensor(pot_table1):std())

In [ ]:
pot_table1 = {}
local number_of_games = 50
local cards_tensor = construct_possible_game_states()
for i = 1,cards_tensor:size(1) do
    for j = 1,number_of_games do
        local cards = distribute_cards(cards_tensor[i])
        tree_eval:play_deterministic_games(tree1,tree2,number_of_plays,cards)   --- --> CHECK NUMBER OF GAMES!
        table.insert(pot_table1,tree_eval.A2_pot)
    end
end
print("player_2 won: ",torch.FloatTensor(pot_table1):mean())
print("player_2 std: ",torch.FloatTensor(pot_table1):std())

In [ ]:
 cardsInv = {}
cardsInv.P1 = cards.P2
cardsInv.P2 = cards.P1
cardsInv.board = cards.board

In [ ]:
pot_table2 = {}
local size_sample = 100
for i = 1,20 do
    tree_eval:play_deterministic_games(tree1,tree2,size_sample,cardsInv)
    table.insert(pot_table2,tree_eval.A2_pot/size_sample)
end
print("player_2 won: ",torch.FloatTensor(pot_table2):mean())
print("player_2 std: ",torch.FloatTensor(pot_table2):std())

In [ ]:
---plot = Plot():histogram(torch.FloatTensor(pot_table1)):draw()

In [ ]:
---plot = Plot():histogram(torch.FloatTensor(pot_table2)):draw()

In [ ]:
x[1]

In [ ]:
number_of_times = 100
interval = 20
range_games = torch.range(0,number_of_times,interval)
pots_won_tensor = torch.FlaoatTensor(1,range_games:size(1))
avg_gain_per_game = torch.FloatTensor(1,range_games:size(1))
range_games[1] = 1
for i = 1,range_games:size(1) do
    print("iteration: ", range_games[i])
    tree_eval:play_all_combinations_n_times(tree_bench,tree_200,range_games[i])
    local pot_won = tree_eval.A2_pot_matrix:sum()/(pot*range_games[i]*60)
    local avg_gain = tree_eval.A2_pot_matrix:sum()/(range_games[i]*60)
    pots_won_tensor[{1,i}] = pot_won
    avg_gain_per_game[{1,i}] = avg_gain
    print("Total number of pots won : ",pot_won)
    print("average amount won per game",avg_gain)
    collectgarbage()
end
